In [81]:
import tensorflow as tf
import pandas as pd
import numpy as np

Here, we define the columns of the training set, and load it from the file.

In [90]:
trainingSetStamp='1560159859969'
training_column_names=["HAND_TYPE",
              "FINGER1_X", "FINGER1_Y", "FINGER1_Z",
              "FINGER2_X", "FINGER2_Y", "FINGER2_Z",
              "FINGER3_X", "FINGER3_Y", "FINGER3_Z",
              "FINGER4_X", "FINGER4_Y", "FINGER4_Z",
              "FINGER5_X", "FINGER5_Y", "FINGER5_Z"]
column_names=["HAND_TYPE",
              "FINGER1_X", "FINGER1_Y", "FINGER1_Z",
              "FINGER2_X", "FINGER2_Y", "FINGER2_Z",
              "FINGER3_X", "FINGER3_Y", "FINGER3_Z",
              "FINGER4_X", "FINGER4_Y", "FINGER4_Z",
              "FINGER5_X", "FINGER5_Y", "FINGER5_Z",
              "INSTRUCTION_LABEL"]
data = pd.read_csv('../../trainingAssets/sets/trainingSet_'+trainingSetStamp+'.txt', sep="|", header=None, names=column_names)

Now, we shuffle the data and partition it into training set and test set

In [91]:
data = data.sample(frac=1).reset_index(drop=True)
test_len = (len(data) * 20)//100;
training_data = data[test_len:]
test_data = data[:test_len]

Here we define the feature columns, describing the data on the data frame. 

TODO: Fix the type: The columns HAND_TYPE and INSTRUCTION_LABEL are int, not float32

In [92]:
feature_columns=[tf.contrib.layers.real_valued_column(column_name, dimension=1, dtype=tf.float32) for column_name in training_column_names]

Format the data as required by Tensorflow

In [93]:
x_train_data = { column_name : np.array(training_data[column_name]) for column_name in training_column_names }

Define and train the model: A neural network classifier, with 5 layers with 5 neurons each, and a possible output of 5 labels.

In [94]:
classifier = tf.estimator.DNNClassifier(
       feature_columns = feature_columns,
       hidden_units = [5, 5],
       n_classes = 5)

# Define the training inputs
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x = x_train_data,
    y = np.array(training_data["INSTRUCTION_LABEL"]).astype(int),
    num_epochs = None,
    shuffle = True)

# Train model.
classifier.train(input_fn = train_input_fn, steps = 2000)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpjkuokdwb', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f38afdbf470>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph wa

Use the previously separated test data to evaluate the accuracy of the model

In [95]:
x_testing_data = { column_name : np.array(test_data[column_name]) for column_name in training_column_names }

# Define the training inputs
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x = x_testing_data,
    y = np.array(test_data["INSTRUCTION_LABEL"]).astype(int),
    num_epochs = 1,
    shuffle = False)

# Evaluate accuracy.
accuracy_score = classifier.evaluate(input_fn=test_input_fn)["accuracy"]

print("Test Accuracy: ", accuracy_score)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-06-10T09:46:27Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpjkuokdwb/model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-06-10-09:46:27
INFO:tensorflow:Saving dict for global step 2000: accuracy = 0.768, average_loss = 0.45445916, global_step = 2000, loss = 56.807396
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 2000: /tmp/tmpjkuokdwb/model.ckpt-2000
Test Accuracy:  0.768


Export and store the trained model

In [96]:
def serving_input_receiver_fn():
    serialized_tf_example = tf.placeholder(dtype = tf.string, shape = [None], name = 'input_tensors')
    receiver_tensors      = {'predictor_inputs' : serialized_tf_example}
    feature_spec          = { column_name : tf.FixedLenFeature([25], tf.float32) for column_name in training_column_names }
    features              = tf.parse_example(serialized_tf_example, feature_spec)
    return tf.estimator.export.ServingInputReceiver(features, receiver_tensors)


model_dir = classifier.export_savedmodel(export_dir_base = "../../trainingAssets/models/"+trainingSetStamp, 
                             serving_input_receiver_fn = serving_input_receiver_fn,
                             as_text = True)
print('Model exported to '+ model_dir.decode())

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: ['serving_default', 'classification']
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predict']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from /tmp/tmpjkuokdwb/model.ckpt-2000
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ../../trainingAssets/models/1560159859969/temp-b'1560160000'/saved_model.pbtxt
Model exported to ../../trainingAssets/models/1560159859969/1560160000
